In [1]:
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize
import xapian
import string
import sys
import time
import glob
from pandas import DataFrame
import spacy
import import_ipynb
from Features_extraction import *

importing Jupyter notebook from Features_extraction.ipynb


In [2]:
import swifter

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
test_file = 'test-unlabelled.json'  

test_set = pd.read_json(open(test_file),orient='index')

In [4]:
test_set.head()

,claim
110000,Raven-Symoné is an Anglican.
73397,Temple of the Dog celebrated the 37th annivers...
16079,Solanum contains plants with ornamental flowers.
114789,A Good Day to Die Hard is part of the Library ...
126871,The Icelandic Coast Guard is also known as Gae...


In [5]:
test_set.reset_index(inplace=True)

In [6]:
test_set.head()

,index,claim
0,110000,Raven-Symoné is an Anglican.
1,73397,Temple of the Dog celebrated the 37th annivers...
2,16079,Solanum contains plants with ornamental flowers.
3,114789,A Good Day to Die Hard is part of the Library ...
4,126871,The Icelandic Coast Guard is also known as Gae...


In [7]:
start = time.time()
# Open the database for searching.
database = xapian.Database("db-index")
# Start an enquire session.
enquire = xapian.Enquire(database)

In [8]:
doc=[]
def get_topk_evidence(df):
    claim_id = df["index"]
    claim = df["claim"]
    query_string = claim
    # Parse the query string to produce a Xapian::Query object.
    qp = xapian.QueryParser()
    stemmer = xapian.Stem("english")
    qp.set_stemmer(stemmer)
    qp.set_database(database)
    qp.set_stemming_strategy(xapian.QueryParser.STEM_SOME)
    query = qp.parse_query(query_string)

    # Find the top 10 results for the query.
    enquire.set_query(query)
    matches = enquire.get_mset(0, 5)

    
    
    for m in matches:
        a = m.document.get_data().decode('utf-8')
        tile_pageid = a.split(' ', 2)[:2]
        text = a.split(' ', 2)[2] 
        if str(tile_pageid[1]).isdigit():
            doc.append(dict(
                claim_id = claim_id,
                claim = claim,
                ir_lookup = a,
                evidence_topk = tile_pageid,
                text_topk =text,
                percentage = m.percent,
                weight = m.weight,            
            ))


In [9]:
nun = test_set.swifter.apply(get_topk_evidence,axis=1)

In [10]:
test_set = pd.DataFrame(doc)

In [11]:
test_set.head()

,claim,claim_id,evidence_topk,ir_lookup,percentage,text_topk,weight
0,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné_-LRB-album-RRB-, 0]",Raven-Symoné_-LRB-album-RRB- 0 Raven-Symoné is...,60,Raven-Symoné is the eponymous fourth studio al...,33.126754
1,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné_discography, 22]",Raven-Symoné_discography 22 Symoné released he...,57,Symoné released her fourth studio album Raven-...,31.608605
2,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné, 0]",Raven-Symoné 0 Raven-Symoné Christina Pearman ...,55,Raven-Symoné Christina Pearman -LRB- -LSB- ˈre...,30.747189
3,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné_discography, 0]",Raven-Symoné_discography 0 The following is a ...,55,The following is a discography for the America...,30.456548
4,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné, 14]","Raven-Symoné 14 As of April 2008 , Raven-Symon...",53,"As of April 2008 , Raven-Symoné has sold 314,0...",29.496223


# Feature Extraction

In [ ]:
jaccard_sim = []
lcs_data = []
spacy_sim = []
cosine_sim = []
ngram = []

    
for i, val in test_set.iterrows():
    claim = val['claim']
    evidence = val['text_topk'] #val['evidence']

    #Jaccard Similarity 
    jacc = jaccardSim(claim, evidence)
    jaccard_sim.append(jacc)
    
    #LCS
    lcs_count = lcs(str(claim), str(evidence))
    lcs_data.append(lcs_count)
    
    #spacy simlarity
    sim = (nlp(claim)).similarity(nlp(evidence))
    spacy_sim.append(sim)
    
    #cosine_sim_tfidf
    document = [claim,evidence]
    cosine_sim.append(cosineSimTfidf(document))
    
    #NGram compare
    ngram_comp = NGram.compare(claim,evidence)
    ngram.append(ngram_comp)
    
    
test_set['jaccard_sim'] = jaccard_sim
test_set['cosine_sim_tfidf'] = cosine_sim
test_set['spaCy_similarity'] = spacy_sim
test_set['lcs'] = lcs_data
test_set['ngram'] = ngram

In [17]:
test_set.head()

,claim,claim_id,evidence_topk,ir_lookup,percentage,text_topk,weight,jaccard_sim,cosine_sim_tfidf,spaCy_similarity,lcs,ngram
0,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné_-LRB-album-RRB-, 0]",Raven-Symoné_-LRB-album-RRB- 0 Raven-Symoné is...,60,Raven-Symoné is the eponymous fourth studio al...,33.126754,0.166667,0.380809,0.697864,2,0.161616
1,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné_discography, 22]",Raven-Symoné_discography 22 Symoné released he...,57,Symoné released her fourth studio album Raven-...,31.608605,0.062500,0.223273,0.595640,1,0.127660
2,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné, 0]",Raven-Symoné 0 Raven-Symoné Christina Pearman ...,55,Raven-Symoné Christina Pearman -LRB- -LSB- ˈre...,30.747189,0.096774,0.277305,0.657899,3,0.092105
3,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné_discography, 0]",Raven-Symoné_discography 0 The following is a ...,55,The following is a discography for the America...,30.456548,0.153846,0.238553,0.751015,1,0.177083
4,Raven-Symoné is an Anglican.,110000,"[Raven-Symoné, 14]","Raven-Symoné 14 As of April 2008 , Raven-Symon...",53,"As of April 2008 , Raven-Symoné has sold 314,0...",29.496223,0.055556,0.135028,0.681647,1,0.110000


In [18]:
test_set.to_csv('test_set_with_features_new.csv') #to csv